In [ ]:
import math
import numpy as np
import sklearn
from sklearn.cluster import KMeans
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns
import pandas as pd
import random
from scipy import stats  #t-test
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from scipy import signal

import xlrd
from itertools import groupby

from statistics import mean
from matplotlib import cm
import csv

from scipy.spatial.distance import cdist
import warnings; warnings.simplefilter('ignore')

In [ ]:
pathfp=r'F:\KyotoUdata\EEG\FP'
filesfp=sorted(os.listdir(pathfp))
pathsf=r'F:\KyotoUdata\EEG\SF'
filessf=sorted(os.listdir(pathsf))
pathre=r'F:\KyotoUdata\EEG\RE'
filesre=sorted(os.listdir(pathre))
pathst=r'F:\KyotoUdata\EEG\ST'
filesst=sorted(os.listdir(pathst))

In [ ]:
datafpm=np.empty((len(filesfp),1),dtype=object)
datasfm=np.empty((len(filessf),1),dtype=object)
datarem=np.empty((len(filesre),1),dtype=object)
datastm=np.empty((len(filesst),1),dtype=object)

In [ ]:
datafpc=np.empty((len(filesfp),1),dtype=object)
datasfc=np.empty((len(filessf),1),dtype=object)
datarec=np.empty((len(filesre),1),dtype=object)
datastc=np.empty((len(filesst),1),dtype=object)

In [ ]:
headfpm=np.empty((len(filesfp),16),dtype=object)
headsfm=np.empty((len(filessf),16),dtype=object)
headrem=np.empty((len(filesre),16),dtype=object)
headstm=np.empty((len(filesst),16),dtype=object)

In [ ]:
headfpc=np.empty((len(filesfp),16),dtype=object)
headsfc=np.empty((len(filessf),16),dtype=object)
headrec=np.empty((len(filesre),16),dtype=object)
headstc=np.empty((len(filesst),16),dtype=object)

In [ ]:
def removespace(listin):
    for i in listin:
        if '' in listin:
            listin.remove('')
    return listin

In [ ]:
def dataread(pathin):
    datap= pd.read_csv(pathin)
    columnlen=len(datap.iloc[:,0])
    head=[]
    with open (pathin) as f:
        test = list(csv.reader(f, delimiter=" "))
        test1=[x for x in test[:][0] if x!='']
        rowlen=len(test1)
    data=np.zeros((rowlen-1,columnlen))
    with open (pathin) as f:
        test = list(csv.reader(f, delimiter=" "))
        for i in range(columnlen):
            tem=[x for x in test[:][i] if x!='']
            data[:,i]=tem[1:]
            head.append(tem[0])
    return head,data

In [ ]:
for i in range(len(filesfp)):
    headfp,datafp=dataread(pathfp+'\\'+filesfp[i])
    headsf,datasf=dataread(pathsf+'\\'+filessf[i])
    headre,datare=dataread(pathre+'\\'+filesre[i])
    headst,datast=dataread(pathst+'\\'+filesst[i])
    #print(headfp[:16],headfp[18:-1])
    headfpc[i]=headfp[:16]
    headfpm[i]=headfp[18:-1]
    headsfc[i]=headsf[:16]
    headsfm[i]=headsf[18:-1]
    headrec[i]=headre[:16]
    headrem[i]=headre[18:-1]
    headstc[i]=headst[:16]
    headstm[i]=headst[18:-1]
    
    datafpc[i]=[datafp[:,:16]]
    datafpm[i]=[datafp[:,18:-1]]
    datasfc[i]=[datasf[:,:16]]
    datasfm[i]=[datasf[:,18:-1]]
    datarec[i]=[datare[:,:16]]
    datarem[i]=[datare[:,18:-1]]
    datastc[i]=[datast[:,:16]]
    datastm[i]=[datast[:,18:-1]]

In [ ]:
datafprawm=np.empty((len(filesfp),1),dtype=object)
datasfrawm=np.empty((len(filessf),1),dtype=object)
datarerawm=np.empty((len(filesre),1),dtype=object)
datastrawm=np.empty((len(filesst),1),dtype=object)

datafprawc=np.empty((len(filesfp),1),dtype=object)
datasfrawc=np.empty((len(filessf),1),dtype=object)
datarerawc=np.empty((len(filesre),1),dtype=object)
datastrawc=np.empty((len(filesst),1),dtype=object)

In [ ]:
def meanall(datain,dataout):
    for i in range(len(datain)):
        dataout[i]=[np.mean(datain[i][0],axis=1)]
    return dataout

In [ ]:
datafprawm=meanall(datafpm,datafprawm)
datasfrawm=meanall(datasfm,datasfrawm)
datarerawm=meanall(datarem,datarerawm)

datafprawc=meanall(datafpc,datafprawc)
datasfrawc=meanall(datasfc,datasfrawc)
datarerawc=meanall(datarec,datarerawc)

In [ ]:
def datacombine(datafpin,datasfin,datarein,datacin):
    for i in range(len(datafpin)):
        datacin[i]=[np.r_[datafpin[i][0],datasfin[i][0],datarein[i][0]]]
    return datacin

In [ ]:
bdatam=np.empty((len(filesfp),1),dtype=object)
bdatac=np.empty((len(filesfp),1),dtype=object)

In [ ]:
bdatam=datacombine(datafprawm,datasfrawm,datarerawm,bdatam)
bdatac=datacombine(datafprawc,datasfrawc,datarerawc,bdatac)

In [ ]:
bdatamdown=np.empty((len(filesfp),1),dtype=object)
bdatacdown=np.empty((len(filesfp),1),dtype=object)

In [ ]:
sampling_rate = 500
downsample_factor = 1  # In this case, downsample by 1 second

In [ ]:
ddatafpm=np.empty((len(filesfp),1),dtype=object)
ddatasfm=np.empty((len(filessf),1),dtype=object)
ddatarem=np.empty((len(filesre),1),dtype=object)
ddatastm=np.empty((len(filesst),1),dtype=object)

ddatafpc=np.empty((len(filesfp),1),dtype=object)
ddatasfc=np.empty((len(filessf),1),dtype=object)
ddatarec=np.empty((len(filesre),1),dtype=object)
ddatastc=np.empty((len(filesst),1),dtype=object)

In [ ]:
def downsample_time_serieso(time_series, sampling_rate, downsample_factor):
    # Calculate the number of samples to average for downsampling
    samples_per_second = sampling_rate
    samples_to_average = int(samples_per_second * downsample_factor)

    # Calculate overlap
    overlap = samples_to_average // 2

    # Calculate the number of windows
    num_windows = (len(time_series) - samples_to_average) // overlap+1

    # Initialize the downsampled time series array
    downsampled_time_series = np.zeros((num_windows, 1))

    # Populate the downsampled time series array using overlapping windows
    for i in range(num_windows):
        start = i * overlap
        end = start + samples_to_average
        window_data = time_series[start:end]# Extract the windowed data
        #print('w',len(window_data))
        downsampled_time_series[i] = np.mean(window_data, axis=0)  # Compute the mean along the window
        #print('d',downsampled_time_series[i])
    return downsampled_time_series

In [ ]:
def downall(dataallin,dataout):
    for i in range(len(filesfp)):
        datain=dataallin[i][0]
        #print(i,datain)
        downin=downsample_time_serieso(datain, sampling_rate, downsample_factor)
        dataout[i]=[downin]
    return dataout

In [ ]:
ddatafpm=downall(datafprawm,ddatafpm)
ddatafpc=downall(datafprawc,ddatafpc)

ddatasfm=downall(datasfrawm,ddatasfm)
ddatasfc=downall(datasfrawc,ddatasfc)

ddatarem=downall(datarerawm,ddatarem)
ddatarec=downall(datarerawc,ddatarec)

In [ ]:
bdatamdown=datacombine(ddatafpm,ddatasfm,ddatarem,bdatamdown)
bdatacdown=datacombine(ddatafpc,ddatasfc,ddatarec,bdatacdown)

In [ ]:
def calculate_max_distance(data1, data2):
    # Convert data series to numpy arrays
    data1 = np.array(data1)
    data2 = np.array(data2)
    
    max_distance = 0
    
    # Calculate the distances between each pair of points from data1 and data2
    for x in data1:
        for y in data2:
            distance = abs(x - y)
            if distance > max_distance:
                max_distance = distance
    
    return max_distance

In [ ]:
def maxall(datainm,datainc):
    maxall=[]
    #minall=[]
    for i in range(len(datainm)):
        maxall.append(calculate_max_distance(datainm[i][0],datainc[i][0]))
        #minall.append(thresholdeach(datainm[i][0],datainc[i][0])[1])
    print(i)
    maxout=np.max(maxall)
    return 0.05*maxout,maxall

In [ ]:
echi,maxchi=maxall(bdatacdown,bdatacdown)
emot,maxmot=maxall(bdatamdown,bdatamdown)
ecrp,maxc=maxall(bdatamdown,bdatacdown)

In [ ]:
def rrcoutcolorall(y,yi,thresholdin,namein): # for the recurrence plot, please define y=yi
    ya=np.array(y)
    yia=np.array(yi)
    matrix=np.zeros((len(ya),len(yia))) #initialize the matrix for the horizontal axis
    matrixx=np.zeros((len(ya),len(yia))) #initialize the matrix for the vertical axis
    matrixc=np.zeros((len(ya),len(yia)))-100 # intitialize the color matrix to distinguish the values on the plots
    for i in range(len(ya)): # go through the y sequency, and find all the equalized value in yi for each time point in y sequence
        ee=cdist(yia,[ya[i,:]])
        indf=np.where(ee<=thresholdin) # if you want to define the tolerance,not y=yi, but |y-yi|<tolerance, please modify this line as indf=np.where(abs(yia-ya[i])<tolerance) 
        iii=indf[0]
        yyy=np.array([np.nan]*len(yia))
        yyyt=np.arange(len(yia))
        xlabel=np.zeros((1,len(ya)))+i
        matrixx[:,i]=xlabel
        yyy=ee.ravel()
        matrixc[:,i]=yyy
        matrix[:,i]=yyyt
    plt.figure(figsize=(6.1,5)) # plot the figure
    #plt.axis('off')
    dx=np.arange(len(ya))
    cmap=cm.get_cmap('hot',11)
    plt.scatter([i for i in matrixx],[j for j in matrix],c=[k for k in matrixc],cmap=cmap,s=1,vmin=0, vmax=int(thresholdin))
    plt.xlim(0,len(ya))
    plt.ylim(0,len(ya))
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    cbar=plt.colorbar()
    tick_font_size = 30
    cbar.ax.tick_params(labelsize=tick_font_size)
    cbar.locator=mpl.ticker.MaxNLocator(nbins=11)
    cbar.update_ticks()
    #pathsave=r'E:\KyotoUdata\EEG\figureout'
    #plt.savefig(pathsave+'\\'+namein)
    plt.show()

In [ ]:
def rrcoutblackall(y,yi,thresholdin,namein): # for the recurrence plot, please define y=yi
    ya=np.array(y)
    yia=np.array(yi)
    matrix=np.zeros((len(ya),len(yia))) #initialize the matrix for the horizontal axis
    matrixx=np.zeros((len(ya),len(yia))) #initialize the matrix for the vertical axis
    matrixc=np.zeros((len(ya),len(yia)))-100 # intitialize the color matrix to distinguish the values on the plots
    for i in range(len(ya)): # go through the y sequency, and find all the equalized value in yi for each time point in y sequence
        ee=cdist(yia,[ya[i,:]])
        indf=np.where(ee<=thresholdin)
        indf0=np.where(ee>thresholdin)# if you want to define the tolerance,not y=yi, but |y-yi|<tolerance, please modify this line as indf=np.where(abs(yia-ya[i])<tolerance) 
        iii=indf[0]
        yyy=np.array([np.nan]*len(yia))
        yyyt=np.arange(len(yia))
        xlabel=np.zeros((1,len(ya)))+i
        matrixx[:,i]=xlabel
        ee[iii]=1
        ee[indf0[0]]=0
        yyy=ee.ravel()
        matrixc[:,i]=yyy
        matrix[:,i]=yyyt
    plt.figure(figsize=(5,5)) # plot the figure
    #plt.axis('off')
    dx=np.arange(len(ya))
    cmap=cm.get_cmap('binary')
    plt.scatter([i for i in matrixx],[j for j in matrix],c=[k for k in matrixc],cmap=cmap,s=1,vmin=0,vmax=1)
    plt.xlim(0,len(ya))
    plt.ylim(0,len(ya))
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    #cbar=plt.colorbar()
    #tick_font_size = 30
    #cbar.ax.tick_params(labelsize=tick_font_size)
    #cbar.locator=mpl.ticker.MaxNLocator(nbins=11)
    #cbar.update_ticks()
    pathsave=r'F:\KyotoUdata\EEG\rps0716'
    plt.savefig(pathsave+'\\'+namein,bbox_inches='tight',pad_inches=0.0)
    plt.show()

In [ ]:
#test1=np.array(bdatamdown[0][0])
#test2=np.array(bdatamdown[0][0])
#print(test1)

In [ ]:
#plt.figure(figsize=(13, 4))
#plt.plot(np.array(bdatac[-1][0]))
#plt.show()
#plt.figure(figsize=(13, 4))
#plt.plot(np.array(datasfc[-1][0]))
#plt.show()
#plt.figure(figsize=(13, 4))
#plt.plot(np.array(bdatacdown[-1][0]))
#plt.show()


In [ ]:
for i in range(len(bdatacdown)):
    print(filesfp[i])
    plt.figure(figsize=(6, 2))
    plt.plot(np.array(bdatacdown[i][0]))
    plt.xlim(0,720)
    plt.ylim(-15,15)
    plt.show()

In [ ]:
#labels=[0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]

In [ ]:
for i in range(len(bdatamdown)):
    print(filesfp[i])
    plt.figure(figsize=(6, 2))
    plt.plot(np.array(bdatamdown[i][0]))
    plt.xlim(0,720)
    plt.ylim(-15,15)
    plt.show()

In [ ]:
categories=['006','007','008','010','011','014','015','018','024','025',
           '026','030','032','033','034','035','037','038','041','042']
#labels=[0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1]
labels=[0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
color_map = {0: 'red', 1: 'blue', 2: 'yellow'}
colors = [color_map[label] for label in labels]
def corrsep(corrin,labelin):
    corr0=corrin[np.where(np.array(labelin)==0)[0]]
    corr1=corrin[np.where(np.array(labelin)==1)[0]]
    return corr0,corr1

In [ ]:
childall0,childall1=corrsep(np.array(bdatacdown),labels)
motherall0,motherall1=corrsep(np.array(bdatamdown),labels)

name0,name1=corrsep(np.array(filesfp),labels)

In [ ]:
print(name0)

In [ ]:
print(name1)

In [ ]:
for i in range(len(childall1)):
    #if i>=1:
        #break
    name='all'+'\\'+'yes'+'\\'+'child'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(childall1[i][0],childall1[i][0],echi,name)

In [ ]:
for i in range(len(motherall1)):
    #if i>=1:
        #break
    name='all'+'\\'+'yes'+'\\'+'mother'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(motherall1[i][0],motherall1[i][0],emot,name)

In [ ]:
for i in range(len(childall0)):
    #if i>=1:
        #break
    name='all'+'\\'+'no'+'\\'+'child'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(childall0[i][0],childall0[i][0],echi,name)

In [ ]:
for i in range(len(motherall0)):
    #if i>=1:
        #break
    name='all'+'\\'+'no'+'\\'+'mother'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(motherall0[i][0],motherall0[i][0],emot,name)

In [ ]:
for i in range(len(motherall0)):
    #if i>=1:
        #break
    name='fp'+'\\'+'no'+'\\'+'mother'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(motherall0[i][0][:240],motherall0[i][0][:240],emot,name)

In [ ]:
for i in range(len(childall0)):
    #if i>=1:
        #break
    name='fp'+'\\'+'no'+'\\'+'child'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(childall0[i][0][:240],childall0[i][0][:240],echi,name)

In [ ]:
for i in range(len(motherall1)):
    #if i>=1:
        #break
    name='fp'+'\\'+'yes'+'\\'+'mother'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(motherall1[i][0][:240],motherall1[i][0][:240],emot,name)

In [ ]:
for i in range(len(childall1)):
    #if i>=1:
        #break
    name='fp'+'\\'+'yes'+'\\'+'child'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(childall1[i][0][:240],childall1[i][0][:240],echi,name)

In [ ]:
for i in range(len(motherall0)):
    #if i>=1:
        #break
    name='sf'+'\\'+'no'+'\\'+'mother'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(motherall0[i][0][240:480],motherall0[i][0][240:480],emot,name)

In [ ]:
for i in range(len(childall0)):
    #if i>=1:
        #break
    name='sf'+'\\'+'no'+'\\'+'child'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(childall0[i][0][240:480],childall0[i][0][240:480],echi,name)

In [ ]:
for i in range(len(motherall1)):
    #if i>=1:
        #break
    name='sf'+'\\'+'yes'+'\\'+'mother'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(motherall1[i][0][240:480],motherall1[i][0][240:480],emot,name)

In [ ]:
for i in range(len(childall1)):
    #if i>=1:
        #break
    name='sf'+'\\'+'yes'+'\\'+'child'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(childall1[i][0][240:480],childall1[i][0][240:480],echi,name)

In [ ]:
for i in range(len(motherall0)):
    #if i>=1:
        #break
    name='re'+'\\'+'no'+'\\'+'mother'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(motherall0[i][0][-240:],motherall0[i][0][-240:],emot,name)

In [ ]:
for i in range(len(childall0)):
    #if i>=1:
        #break
    name='re'+'\\'+'no'+'\\'+'child'+'\\'+name0[i][:3]+'.jpg'
    rrcoutblackall(childall0[i][0][-240:],childall0[i][0][-240:],echi,name)

In [ ]:
for i in range(len(motherall1)):
    #if i>=1:
        #break
    name='re'+'\\'+'yes'+'\\'+'mother'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(motherall1[i][0][-240:],motherall1[i][0][-240:],emot,name)

In [ ]:
for i in range(len(childall1)):
    #if i>=1:
        #break
    name='re'+'\\'+'yes'+'\\'+'child'+'\\'+name1[i][:3]+'.jpg'
    rrcoutblackall(childall1[i][0][-240:],childall1[i][0][-240:],echi,name)

In [ ]:
#labels=[0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]

In [ ]:
for i in range(len(bdatamdown)):
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(bdatamdown[i][0],bdatamdown[i][0],emot,name)

In [ ]:
for i in range(len(bdatamdown)):
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(bdatamdown[i][0][240:480],bdatamdown[i][0][240:480],emot,name)

In [ ]:
for i in range(len(bdatamdown)):
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(bdatamdown[i][0],bdatacdown[i][0],ecrp,name)

In [ ]:
for i in range(len(bdatacdown)):
    #if i>=1:
        #break
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(bdatacdown[i][0][240:480],bdatacdown[i][0][240:480],echi,name)

In [ ]:
#for i in range(len(bdatacdown)):
    #if i>=1:
        #break
    #name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    #rrcoutcolorall(bdatacdown[i][0],bdatacdown[i][0],echi,name)

In [ ]:
#for i in range(len(bdatamdown)):
    #name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    #rrcoutcolorall(bdatamdown[i][0],bdatamdown[i][0],emot,name)

In [ ]:
#for i in range(len(bdatamdown)):
    #name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    #rrcoutcolorall(bdatamdown[i][0],bdatacdown[i][0],ecrp,name)

In [ ]:
def rrcoutmatrix(y,yi,thresholdin): # for the recurrence plot, please define y=yi
    ya=np.array(y)
    yia=np.array(yi)
    matrix=np.zeros((len(ya),len(yia))) #initialize the matrix for the horizontal axis
    matrixx=np.zeros((len(ya),len(yia))) #initialize the matrix for the vertical axis
    matrixc=np.zeros((len(ya),len(yia)))-100 # intitialize the color matrix to distinguish the values on the plots
    for i in range(len(ya)): # go through the y sequency, and find all the equalized value in yi for each time point in y sequence
        ee=cdist(yia,[ya[i,:]])
        indf=np.where(ee<=thresholdin) # if you want to define the tolerance,not y=yi, but |y-yi|<tolerance, please modify this line as indf=np.where(abs(yia-ya[i])<tolerance) 
        iii=indf[0]
        yyy=np.zeros(len(yia))
        yyyt=np.array([np.nan]*len(yia))
        xlabel=np.zeros((1,len(ya)))+i
        matrixx[:,i]=xlabel
        if len(iii)!=0:
            #yyy[iii]=np.mean(ya[i,:])
            yyy[iii]=1 # recurrence points
            yyyt[iii]=iii
            matrix[:,i]=yyyt
            matrixc[:,i]=yyy
    for j in range(len(matrixc[:,0])):
        inddd=np.where(matrixc[j,:]==-100)
        inddd0=np.array(inddd[0])
        matrixc[j,inddd0]=0
    return matrixc

In [ ]:
def recurrence_quantification_analysis(recurrence_matrix, minimum_diagonal_line_length, minimum_vertical_line_length, minimum_white_vertical_line_length):
    # Calculating the number of states - N
    number_of_vectors = recurrence_matrix.shape[0]
    
    # Calculating the diagonal frequency distribution - P(l)
    diagonal_frequency_distribution = np.zeros(number_of_vectors+1)
    for i in range(number_of_vectors-1, -1, -1):
        diagonal_line_length = 0
        for j in range(0, number_of_vectors-i):
            if recurrence_matrix[i+j,j] == 1:
                diagonal_line_length += 1
                if j == (number_of_vectors-i-1):
                    diagonal_frequency_distribution[diagonal_line_length] += 1.0
            else:
                if diagonal_line_length != 0:
                    diagonal_frequency_distribution[diagonal_line_length] += 1.0
                    diagonal_line_length = 0
    for k in range(1,number_of_vectors):
        diagonal_line_length = 0
        for i in range(number_of_vectors-k):
            j = i + k
            if recurrence_matrix[i,j] == 1:
                diagonal_line_length += 1
                if j == (number_of_vectors-1):
                    diagonal_frequency_distribution[diagonal_line_length] += 1.0
            else:
                if diagonal_line_length != 0:
                    diagonal_frequency_distribution[diagonal_line_length] += 1.0
                    diagonal_line_length = 0

    # Calculating the vertical frequency distribution - P(v)
    vertical_frequency_distribution = np.zeros(number_of_vectors+1)
    for i in range(number_of_vectors):
        vertical_line_length = 0
        for j in range(number_of_vectors):
            if recurrence_matrix[i,j] == 1:
                vertical_line_length += 1
                if j == (number_of_vectors-1):
                    vertical_frequency_distribution[vertical_line_length] += 1.0
            else:
                if vertical_line_length != 0:
                    vertical_frequency_distribution[vertical_line_length] += 1.0
                    vertical_line_length = 0

    # Calculating the white vertical frequency distribution - P(w)
    white_vertical_frequency_distribution = np.zeros(number_of_vectors+1)
    for i in range(number_of_vectors):
        white_vertical_line_length = 0
        for j in range(number_of_vectors):
            if recurrence_matrix[i,j] == 0:
                white_vertical_line_length += 1
                if j == (number_of_vectors-1):
                    white_vertical_frequency_distribution[white_vertical_line_length] += 1.0
            else:
                if white_vertical_line_length != 0:
                    white_vertical_frequency_distribution[white_vertical_line_length] += 1.0
                    white_vertical_line_length = 0

    # Calculating the recurrence rate - RR
    recurrence_rate = np.float(np.sum(recurrence_matrix))/np.power(number_of_vectors, 2)

    # Calculating the determinism - DET
    numerator = np.sum([l * diagonal_frequency_distribution[l] for l in range(minimum_diagonal_line_length, number_of_vectors)])
    denominator = np.sum([l * diagonal_frequency_distribution[l] for l in range(1, number_of_vectors)])
    determinism = numerator / denominator

    # Calculating the average diagonal line length - L
    numerator = np.sum([l * diagonal_frequency_distribution[l] for l in range(minimum_diagonal_line_length, number_of_vectors)])
    denominator = np.sum([diagonal_frequency_distribution[l] for l in range(minimum_diagonal_line_length, number_of_vectors)])
    average_diagonal_line_length = numerator / denominator

    # Calculating the longest diagonal line length - Lmax
    for l in range(number_of_vectors-1, 0, -1):
        if diagonal_frequency_distribution[l] != 0:
            longest_diagonal_line_length = l
            break

    # Calculating the  divergence - DIV
    divergence = 1. / longest_diagonal_line_length

    # Calculating the entropy diagonal lines - Lentr
    sum_diagonal_frequency_distribution = np.float(np.sum(diagonal_frequency_distribution[minimum_diagonal_line_length:-1]))
    entropy_diagonal_lines = 0
    for l in range(minimum_diagonal_line_length, number_of_vectors):
        if diagonal_frequency_distribution[l] != 0:
            entropy_diagonal_lines +=  (diagonal_frequency_distribution[l]/sum_diagonal_frequency_distribution) * np.log(diagonal_frequency_distribution[l]/sum_diagonal_frequency_distribution)
    entropy_diagonal_lines *= -1

    # Calculating the ratio determinism_recurrence - DET/RR
    ratio_determinism_recurrence_rate = determinism / recurrence_rate

    # Calculating the laminarity - LAM
    numerator = np.sum([v * vertical_frequency_distribution[v] for v in range(minimum_vertical_line_length, number_of_vectors+1)])
    denominator = np.sum([v * vertical_frequency_distribution[v] for v in range(1, number_of_vectors+1)])
    laminarity = numerator / denominator

    # Calculating the average vertical line length - V
    numerator = np.sum([v * vertical_frequency_distribution[v] for v in range(minimum_vertical_line_length, number_of_vectors+1)])
    denominator = np.sum([vertical_frequency_distribution[v] for v in range(minimum_vertical_line_length, number_of_vectors+1)])
    average_vertical_line_length = numerator / denominator

    # Calculating the longest vertical line length - Vmax
    for v in range(number_of_vectors, 0, -1):
        if vertical_frequency_distribution[v] != 0:
            longest_vertical_line_length = v
            break

    # Calculating the entropy vertical lines - Ventr
    sum_vertical_frequency_distribution = np.float(np.sum(vertical_frequency_distribution[minimum_vertical_line_length:]))
    entropy_vertical_lines = 0
    for v in range(minimum_vertical_line_length, number_of_vectors+1):
        if vertical_frequency_distribution[v] != 0:
            entropy_vertical_lines +=  (vertical_frequency_distribution[v]/sum_vertical_frequency_distribution) * np.log(vertical_frequency_distribution[v]/sum_vertical_frequency_distribution)
    entropy_vertical_lines *= -1

    # Calculatint the ratio laminarity_determinism - LAM/DET
    ratio_laminarity_determinism = laminarity / determinism

    # Calculating the average white vertical line length - W
    numerator = np.sum([w * white_vertical_frequency_distribution[w] for w in range(minimum_white_vertical_line_length, number_of_vectors+1)])
    denominator = np.sum([white_vertical_frequency_distribution[w] for w in range(minimum_white_vertical_line_length, number_of_vectors+1)])
    average_white_vertical_line_length = numerator / denominator

    # Calculating the longest white vertical line length - Wmax
    for w in range(number_of_vectors, 0, -1):
        if white_vertical_frequency_distribution[w] != 0:
            longest_white_vertical_line_length = w
            #break
        else:
            longest_white_vertical_line_length=0

    # Calculating the entropy white vertical lines - Wentr
    sum_white_vertical_frequency_distribution = np.float(np.sum(white_vertical_frequency_distribution[minimum_white_vertical_line_length:]))
    entropy_white_vertical_lines = 0
    for w in range(minimum_white_vertical_line_length, number_of_vectors+1):
        if white_vertical_frequency_distribution[w] != 0:
            entropy_white_vertical_lines +=  (white_vertical_frequency_distribution[w]/sum_white_vertical_frequency_distribution) * np.log(white_vertical_frequency_distribution[w]/sum_white_vertical_frequency_distribution)
    entropy_white_vertical_lines *= -1
    outputall=np.c_[recurrence_rate, determinism, average_diagonal_line_length, longest_diagonal_line_length, divergence, entropy_diagonal_lines, laminarity, average_vertical_line_length, longest_vertical_line_length, entropy_vertical_lines, average_white_vertical_line_length, longest_white_vertical_line_length, entropy_white_vertical_lines, ratio_determinism_recurrence_rate, ratio_laminarity_determinism]

    return diagonal_frequency_distribution, vertical_frequency_distribution, white_vertical_frequency_distribution,outputall

In [ ]:
def jointrpplot(matrix1,matrix2,namein):
    lenm=len(matrix1[:,0])
    recurrence_matrix_1=np.zeros((lenm,lenm))
    labelmatrix=np.zeros((lenm,lenm))
    labelmatrixy=np.zeros((lenm,lenm))
    for ii in range(lenm):
        labelmatrix_t=np.zeros(lenm)+ii
        labelmatrix[:,ii]=labelmatrix_t
    for i in range(lenm):
        for j in range(i, lenm, 1):
            if matrix1[i,j] == 1 and matrix2[i,j] == 1:
                recurrence_matrix_1[i,j] = recurrence_matrix_1[j,i] = 1
            else:
                recurrence_matrix_1[i,j] = recurrence_matrix_1[j,i] = 0
    for k in range(lenm):
        ind1=np.where(recurrence_matrix_1[:,k]==1)[0]
        labelmatrixy_t=np.array([np.nan]*lenm)
        labelmatrixy_t[ind1]=ind1
        labelmatrixy[:,k]=labelmatrixy_t
        #print(ind1)
    output=recurrence_matrix_1.astype(int)
    plt.figure(figsize=(5,5)) # plot the figure
    #plt.axis('off')
    plt.scatter([iii for iii in labelmatrix],[jjj for jjj in labelmatrixy],c=[kkk for kkk in output],cmap='binary',s=1,vmin=0,vmax=1)
    plt.xlim(0,len(matrix1[:,0]))
    plt.ylim(0,len(matrix2[:,0]))
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    pathsave=r'F:\KyotoUdata\EEG\jrps0716'
    plt.savefig(pathsave+'\\'+namein,bbox_inches='tight',pad_inches=0.0)
    plt.show()
    return output, labelmatrix, labelmatrixy

In [ ]:
def jointrpmatrix(matrix1,matrix2):
    lenm=len(matrix1[:,0])
    recurrence_matrix_1=np.zeros((lenm,lenm))
    labelmatrix=np.zeros((lenm,lenm))
    labelmatrixy=np.zeros((lenm,lenm))
    for ii in range(lenm):
        labelmatrix_t=np.zeros(lenm)+ii
        labelmatrix[:,ii]=labelmatrix_t
    for i in range(lenm):
        for j in range(i, lenm, 1):
            if matrix1[i,j] == 1 and matrix2[i,j] == 1:
                recurrence_matrix_1[i,j] = recurrence_matrix_1[j,i] = 1
            else:
                recurrence_matrix_1[i,j] = recurrence_matrix_1[j,i] = 0
    for k in range(lenm):
        ind1=np.where(recurrence_matrix_1[:,k]==1)[0]
        labelmatrixy_t=np.array([np.nan]*lenm)
        labelmatrixy_t[ind1]=ind1
        labelmatrixy[:,k]=labelmatrixy_t
        #print(ind1)
    output=recurrence_matrix_1.astype(int)
    return output, labelmatrix, labelmatrixy

In [ ]:
#labels=[10, 20, 31, 40, 50, 60, 72, 81, 91, 102, 111, 120, 131, 140,  
# 151,  160,  171,  180,  191,  201]

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
outc=[]
outm=[]
outr=[]
outj=[]
for i in range(len(bdatacdown)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixc=rrcoutmatrix(bdatacdown[i][0],bdatacdown[i][0],echi)
    matrixm=rrcoutmatrix(bdatamdown[i][0],bdatamdown[i][0],emot)
    matrixr=rrcoutmatrix(bdatamdown[i][0],bdatacdown[i][0],ecrp)
    matrixj, labelmatrix, labelmatrixy=jointrpplot(matrixm,matrixc,name)
    dfc, vfc, wvfc, outputc=recurrence_quantification_analysis(matrixc, mdl,mvl, mwvl)
    outc.append(outputc)
    dfm, vfm, wvfm, outputm=recurrence_quantification_analysis(matrixm, mdl,mvl, mwvl)
    outm.append(outputm)
    dfr, vfr, wvfr, outputr=recurrence_quantification_analysis(matrixr, mdl,mvl, mwvl)
    outr.append(outputr)
    dfj, vfj, wvfj, outputj=recurrence_quantification_analysis(matrixj, mdl,mvl, mwvl)
    outj.append(outputj)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
outcfp=[]
outmfp=[]
outrfp=[]
outjfp=[]
for i in range(len(bdatacdown)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(bdatacdown[i][0][240:480],bdatacdown[i][0][240:480],echi)
    matrixmfp=rrcoutmatrix(bdatamdown[i][0][240:480],bdatamdown[i][0][240:480],emot)
    matrixrfp=rrcoutmatrix(bdatamdown[i][0][240:480],bdatacdown[i][0][240:480],ecrp)
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)
    #dfc, vfc, wvfc, outputcfp=recurrence_quantification_analysis(matrixcfp, mdl,mvl, mwvl)
    #outcfp.append(outputcfp)
    #dfm, vfm, wvfm, outputmfp=recurrence_quantification_analysis(matrixmfp, mdl,mvl, mwvl)
    #outmfp.append(outputmfp)
    #dfr, vfr, wvfr, outputrfp=recurrence_quantification_analysis(matrixrfp, mdl,mvl, mwvl)
    #outrfp.append(outputrfp)
    #dfj, vfj, wvfj, outputjfp=recurrence_quantification_analysis(matrixjfp, mdl,mvl, mwvl)
    #outjfp.append(outputjfp)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall0)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall0[i][0],childall0[i][0],echi)
    matrixmfp=rrcoutmatrix(motherall0[i][0],motherall0[i][0],emot)
    matrixrfp=rrcoutmatrix(motherall0[i][0],childall0[i][0],ecrp)
    name='all'+'\\'+'no'+'\\'+name0[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall1)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall1[i][0],childall1[i][0],echi)
    matrixmfp=rrcoutmatrix(motherall1[i][0],motherall1[i][0],emot)
    matrixrfp=rrcoutmatrix(motherall1[i][0],childall1[i][0],ecrp)
    name='all'+'\\'+'yes'+'\\'+name1[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall0)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall0[i][0][:240],childall0[i][0][:240],echi)
    matrixmfp=rrcoutmatrix(motherall0[i][0][:240],motherall0[i][0][:240],emot)
    matrixrfp=rrcoutmatrix(motherall0[i][0][:240],childall0[i][0][:240],ecrp)
    name='fp'+'\\'+'no'+'\\'+name0[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall0)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall0[i][0][240:480],childall0[i][0][240:480],echi)
    matrixmfp=rrcoutmatrix(motherall0[i][0][240:480],motherall0[i][0][240:480],emot)
    matrixrfp=rrcoutmatrix(motherall0[i][0][240:480],childall0[i][0][240:480],ecrp)
    name='sf'+'\\'+'no'+'\\'+name0[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall0)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall0[i][0][-240:],childall0[i][0][-240:],echi)
    matrixmfp=rrcoutmatrix(motherall0[i][0][-240:],motherall0[i][0][-240:],emot)
    matrixrfp=rrcoutmatrix(motherall0[i][0][-240:],childall0[i][0][-240:],ecrp)
    name='re'+'\\'+'no'+'\\'+name0[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall1)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall1[i][0][:240],childall1[i][0][:240],echi)
    matrixmfp=rrcoutmatrix(motherall1[i][0][:240],motherall1[i][0][:240],emot)
    matrixrfp=rrcoutmatrix(motherall1[i][0][:240],childall1[i][0][:240],ecrp)
    name='fp'+'\\'+'yes'+'\\'+name1[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall1)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall1[i][0][240:480],childall1[i][0][240:480],echi)
    matrixmfp=rrcoutmatrix(motherall1[i][0][240:480],motherall1[i][0][240:480],emot)
    matrixrfp=rrcoutmatrix(motherall1[i][0][240:480],childall1[i][0][240:480],ecrp)
    name='sf'+'\\'+'yes'+'\\'+name1[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
for i in range(len(childall1)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcfp=rrcoutmatrix(childall1[i][0][-240:],childall1[i][0][-240:],echi)
    matrixmfp=rrcoutmatrix(motherall1[i][0][-240:],motherall1[i][0][-240:],emot)
    matrixrfp=rrcoutmatrix(motherall1[i][0][-240:],childall1[i][0][-240:],ecrp)
    name='re'+'\\'+'yes'+'\\'+name1[i][:3]+'.jpg'
    matrixjfp, labelmatrixfp, labelmatrixyfp=jointrpplot(matrixmfp,matrixcfp,name)

In [ ]:
len(bdatacdown[0][0][-240:])

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
outcsf=[]
outmsf=[]
outrsf=[]
outjsf=[]
for i in range(len(bdatacdown)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcsf=rrcoutmatrix(bdatacdown[i][0][240:480],bdatacdown[i][0][240:480],echi)
    matrixmsf=rrcoutmatrix(bdatamdown[i][0][240:480],bdatamdown[i][0][240:480],emot)
    matrixrsf=rrcoutmatrix(bdatamdown[i][0][240:480],bdatacdown[i][0][240:480],ecrp)
    matrixjsf, labelmatrixsf, labelmatrixysf=jointrpmatrix(matrixmsf,matrixcsf)
    dfc, vfc, wvfc, outputcsf=recurrence_quantification_analysis(matrixcsf, mdl,mvl, mwvl)
    outcsf.append(outputcsf)
    dfm, vfm, wvfm, outputmsf=recurrence_quantification_analysis(matrixmsf, mdl,mvl, mwvl)
    outmsf.append(outputmsf)
    dfr, vfr, wvfr, outputrsf=recurrence_quantification_analysis(matrixrsf, mdl,mvl, mwvl)
    outrsf.append(outputrsf)
    dfj, vfj, wvfj, outputjsf=recurrence_quantification_analysis(matrixjsf, mdl,mvl, mwvl)
    outjsf.append(outputjsf)

In [ ]:
mdl, mvl, mwvl=5,5,5 # these are manually difined
outcre=[]
outmre=[]
outrre=[]
outjre=[]
for i in range(len(bdatacdown)):
    #if i>=3:
        #break
    name='jrp'+'\\'+filesfp[i][:3]+'.jpg'
    matrixcre=rrcoutmatrix(bdatacdown[i][0][-240:],bdatacdown[i][0][-240:],echi)
    matrixmre=rrcoutmatrix(bdatamdown[i][0][-240:],bdatamdown[i][0][-240:],emot)
    matrixrre=rrcoutmatrix(bdatamdown[i][0][-240:],bdatacdown[i][0][-240:],ecrp)
    matrixjre, labelmatrixre, labelmatrixyre=jointrpmatrix(matrixmre,matrixcre)
    dfc, vfc, wvfc, outputcre=recurrence_quantification_analysis(matrixcre, mdl,mvl, mwvl)
    outcre.append(outputcre)
    dfm, vfm, wvfm, outputmre=recurrence_quantification_analysis(matrixmre, mdl,mvl, mwvl)
    outmre.append(outputmre)
    dfr, vfr, wvfr, outputrre=recurrence_quantification_analysis(matrixrre, mdl,mvl, mwvl)
    outrre.append(outputrre)
    dfj, vfj, wvfj, outputjre=recurrence_quantification_analysis(matrixjre, mdl,mvl, mwvl)
    outjre.append(outputjre)

In [ ]:
categories=['006','007','008','010','011','014','015','018','024','025',
           '026','030','032','033','034','035','037','038','041','042']
#labels=[0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1]
labels=[0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
color_map = {0: 'red', 1: 'blue', 2: 'yellow'}
colors = [color_map[label] for label in labels]

In [ ]:
def corrsep(corrin,labelin):
    corr0=corrin[np.where(np.array(labelin)==0)[0]]
    corr1=corrin[np.where(np.array(labelin)==1)[0]]
    return corr0,corr1

In [ ]:
corr0allc,corr1allc=corrsep(np.array(outc),labels)
corr0allm,corr1allm=corrsep(np.array(outm),labels)
corr0allr,corr1allr=corrsep(np.array(outr),labels)
corr0allj,corr1allj=corrsep(np.array(outj),labels)

In [ ]:
corr0allcfp,corr1allcfp=corrsep(np.array(outcfp),labels)
corr0allmfp,corr1allmfp=corrsep(np.array(outmfp),labels)
corr0allrfp,corr1allrfp=corrsep(np.array(outrfp),labels)
corr0alljfp,corr1alljfp=corrsep(np.array(outjfp),labels)

In [ ]:
corr0allcsf,corr1allcsf=corrsep(np.array(outcsf),labels)
corr0allmsf,corr1allmsf=corrsep(np.array(outmsf),labels)
corr0allrsf,corr1allrsf=corrsep(np.array(outrsf),labels)
corr0alljsf,corr1alljsf=corrsep(np.array(outjsf),labels)

In [ ]:
corr0allcre,corr1allcre=corrsep(np.array(outcre),labels)
corr0allmre,corr1allmre=corrsep(np.array(outmre),labels)
corr0allrre,corr1allrre=corrsep(np.array(outrre),labels)
corr0alljre,corr1alljre=corrsep(np.array(outjre),labels)

In [ ]:
child0,child1=corrsep(np.array(bdatacdown),labels)

In [ ]:
for i in range(len(child0)):
    #if i>=1:
        #break
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(child0[i][0][240:480],child0[i][0][240:480],echi,name)

In [ ]:
for i in range(len(child1)):
    #if i>=1:
        #break
    name='infant'+'\\'+filesfp[i][:3]+'.jpg'
    rrcoutblackall(child1[i][0][240:480],child1[i][0][240:480],echi,name)

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
#outputall=np.c_[recurrence_rate, determinism, average_diagonal_line_length, longest_diagonal_line_length, divergence, entropy_diagonal_lines, laminarity, average_vertical_line_length, longest_vertical_line_length, entropy_vertical_lines, average_white_vertical_line_length, longest_white_vertical_line_length, entropy_white_vertical_lines, ratio_determinism_recurrence_rate, ratio_laminarity_determinism]

In [ ]:
plt.plot(np.array(outc).reshape(20,15)[:,0])

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allc.reshape(9,15)[:,i], corr1allc.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allc.reshape(9,15)[:,i], corr1allc.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allcfp.reshape(9,15)[:,i], corr1allcfp.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allcfp.reshape(9,15)[:,i], corr1allcfp.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allcsf.reshape(9,15)[:,i], corr1allcsf.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allcsf.reshape(9,15)[:,i], corr1allcsf.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allcre.reshape(9,15)[:,i], corr1allcre.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allcre.reshape(9,15)[:,i], corr1allcre.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allm.reshape(9,15)[:,i], corr1allm.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allm.reshape(9,15)[:,i], corr1allm.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allmfp.reshape(9,15)[:,i], corr1allmfp.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allmfp.reshape(9,15)[:,i], corr1allmfp.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allmsf.reshape(9,15)[:,i], corr1allmsf.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allmsf.reshape(9,15)[:,i], corr1allmsf.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allmre.reshape(9,15)[:,i], corr1allmre.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allmre.reshape(9,15)[:,i], corr1allmre.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allr.reshape(9,15)[:,i], corr1allr.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allr.reshape(9,15)[:,i], corr1allr.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allrfp.reshape(9,15)[:,i], corr1allrfp.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allrfp.reshape(9,15)[:,i], corr1allrfp.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allrsf.reshape(9,15)[:,i], corr1allrsf.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allrsf.reshape(9,15)[:,i], corr1allrsf.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allrre.reshape(9,15)[:,i], corr1allrre.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allrre.reshape(9,15)[:,i], corr1allrre.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0allj.reshape(9,15)[:,i], corr1allj.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0allj.reshape(9,15)[:,i], corr1allj.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0alljfp.reshape(9,15)[:,i], corr1alljfp.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0alljfp.reshape(9,15)[:,i], corr1alljfp.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0alljsf.reshape(9,15)[:,i], corr1alljsf.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0alljsf.reshape(9,15)[:,i], corr1alljsf.reshape(9,15)[:,i])
    print(p_value)

In [ ]:
for i in range(10):
    title=['RR','DET','ADL','LDL','DEV','EDL','LMR','AVL','LVL','EVL']
    plt.boxplot([corr0alljre.reshape(9,15)[:,i], corr1alljre.reshape(9,15)[:,i]], labels=['No', 'Yes'])
    plt.xlabel('Group')
    plt.title(title[i])
    plt.grid(True)
    plt.show()
    t_statistic, p_value = ttest_ind(corr0alljre.reshape(9,15)[:,i], corr1alljre.reshape(9,15)[:,i])
    print(p_value)